# Sex Prediction with `FLAML`

In [1]:
from brainprint.recon_all.results import load_results
from flaml import AutoML
from sklearn.exceptions import ConvergenceWarning
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import joblib
import numpy as np
import ray
import warnings
warnings.filterwarnings("ignore", module="sklearn", category=ConvergenceWarning)
np.random.seed(0)
ray.init(log_to_driver=False)

2022-07-03 09:31:29,979	INFO services.py:1470 -- View the Ray dashboard at http://127.0.0.1:8265


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.9.2', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '10.12.8.135', 'raylet_ip_address': '10.12.8.135', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-07-03_09-31-27_817980_1695758/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-07-03_09-31-27_817980_1695758/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-07-03_09-31-27_817980_1695758', 'metrics_export_port': 50721, 'gcs_address': '10.12.8.135:57421', 'address': '10.12.8.135:57421', 'node_id': 'aadebd8bdb97e8579825c900e6ccb25f70581a8a009d6e27e67a7d2b'})

In [3]:
results = load_results()

[2022-07-03 09:29:06,267] Successfully read 4331 recon-all execution results from /home/zvi/Projects/brainprint/data/results.csv.
[2022-07-03 09:29:06,281] Successfully read 4331 recon-all execution configurations from /home/zvi/Projects/brainprint/data/configurations.csv.
[2022-07-03 09:29:06,324] Successfully read 4331 scan research context and metadata from /home/zvi/Projects/brainprint/data/context.csv.
[2022-07-03 09:29:06,571] Filtering 4331 recon-all results
[2022-07-03 09:29:06,590] 1919/4331 recon-all results matching [<Protocol.BASE: 'The Base (corrected)'>] detected.
[2022-07-03 09:29:06,612] PARTIAL ATLAS RESULTS! Selecting Atlas.DESTRIEUX atlas results from 1919 recon-all runs returned 1917 rows (2 runs dropped).
[2022-07-03 09:29:06,618] 1336/1917 recon-all results matching ['Default', 'With T2', 'With FLAIR'] detected.
[2022-07-03 09:29:06,622] Successfully selected 654 runs from 218 scans with all 3 recon-all execution configuration results.
[2022-07-03 09:29:06,633] 79

In [3]:
automl_settings = {
    "task": "classification",
    "time_budget": 5 * 60 * 60,
    # "time_budget": 60,    
    "metric": "roc_auc",    
    "log_file_name": "flaml.log",
    "n_jobs": 16,
    "seed": 0,
    "verbose": 2,
    "mem_thres": 4e10,
    "use_ray": True,
    "model_history": True,
}
pipeline_settings = {f"automl__{key}": value for key, value in automl_settings.items()}
models = {}
scores = {}
for execution_configuration in results.configuration:
    X_train, X_test, y_train, y_test = results.split(execution_configuration=execution_configuration, 
                                                     all_configurations=True, 
                                                     single_mode="last", 
                                                     test_size=0.2, 
                                                     random_state=0)
        
    scaler = StandardScaler()
    automl = AutoML()
    steps = [("scaling", scaler), ("automl", automl)]
    pipeline = Pipeline(steps)
    pipeline.fit(X_train, y_train, **pipeline_settings)
    models[execution_configuration] = pipeline
    scores[execution_configuration] = pipeline.score(X_test, y_test)

[2022-06-28 17:27:29,301] Successfully selected 654 runs from 218 scans with all 3 recon-all execution configuration results.
[2022-06-28 17:27:29,317] Successfully selected 402 runs from a total of 134 scans belonging to 134 subjects.
[2022-06-28 17:27:29,319] Filtering 1336 recon-all results
[2022-06-28 17:27:29,325] 813/1336 recon-all results matching ['Default Configuration'] detected.
[2022-06-28 17:27:29,328] Successfully selected 107/813 runs from a total of 107 scans belonging to 107 subjects.
[2022-06-28 17:27:29,329] 107/1336 recon-all runs selected.
[2022-06-28 17:27:29,329] Filtering 1336 recon-all results
[2022-06-28 17:27:29,339] 813/1336 recon-all results matching ['Default Configuration'] detected.
[2022-06-28 17:27:29,344] Successfully selected 27/813 runs from a total of 27 scans belonging to 27 subjects.
[2022-06-28 17:27:29,344] 27/1336 recon-all runs selected.


[I 2022-06-28 17:27:29,375] A new study created in memory with name: optuna
2022-06-28 17:27:29,382	WARNING function_runner.py:603 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
[I 2022-06-28 17:27:29,389] A new study created in memory with name: optuna
2022-06-28 22:27:33,238	INFO stopper.py:363 -- Reached timeout of 18000 seconds. Stopping all trials.
[flaml.automl: 06-28 22:28:18] {2648} WARNING - Time taken to find the best model is 75% of the provided time budget and not all estimators' hyperparameter search converged. Consider increasing the time budget.


[2022-06-28 22:28:18,006] Time taken to find the best model is 75% of the provided time budget and not all estimators' hyperparameter search converged. Consider increasing the time budget.
[2022-06-28 22:28:18,025] Successfully selected 654 runs from 218 scans with all 3 recon-all execution configuration results.
[2022-06-28 22:28:18,038] Successfully selected 402 runs from a total of 134 scans belonging to 134 subjects.
[2022-06-28 22:28:18,040] Filtering 1336 recon-all results
[2022-06-28 22:28:18,045] 254/1336 recon-all results matching ['With T2'] detected.
[2022-06-28 22:28:18,049] Successfully selected 107/254 runs from a total of 107 scans belonging to 107 subjects.
[2022-06-28 22:28:18,050] 107/1336 recon-all runs selected.
[2022-06-28 22:28:18,051] Filtering 1336 recon-all results
[2022-06-28 22:28:18,056] 254/1336 recon-all results matching ['With T2'] detected.
[2022-06-28 22:28:18,059] Successfully selected 27/254 runs from a total of 27 scans belonging to 27 subjects.
[202

[I 2022-06-28 22:28:18,085] A new study created in memory with name: optuna
[I 2022-06-28 22:28:18,094] A new study created in memory with name: optuna
2022-06-29 03:28:21,862	INFO stopper.py:363 -- Reached timeout of 18000 seconds. Stopping all trials.


[2022-06-29 03:28:50,616] Successfully selected 654 runs from 218 scans with all 3 recon-all execution configuration results.
[2022-06-29 03:28:50,637] Successfully selected 402 runs from a total of 134 scans belonging to 134 subjects.
[2022-06-29 03:28:50,641] Filtering 1336 recon-all results
[2022-06-29 03:28:50,646] 269/1336 recon-all results matching ['With FLAIR'] detected.
[2022-06-29 03:28:50,653] Successfully selected 107/269 runs from a total of 107 scans belonging to 107 subjects.
[2022-06-29 03:28:50,655] 107/1336 recon-all runs selected.
[2022-06-29 03:28:50,655] Filtering 1336 recon-all results
[2022-06-29 03:28:50,660] 269/1336 recon-all results matching ['With FLAIR'] detected.
[2022-06-29 03:28:50,666] Successfully selected 27/269 runs from a total of 27 scans belonging to 27 subjects.
[2022-06-29 03:28:50,669] 27/1336 recon-all runs selected.


[I 2022-06-29 03:28:50,700] A new study created in memory with name: optuna
[I 2022-06-29 03:28:50,707] A new study created in memory with name: optuna
2022-06-29 08:28:53,109	INFO stopper.py:363 -- Reached timeout of 18000 seconds. Stopping all trials.


In [4]:
scores

{<ExecutionConfiguration.DEFAULT: 'Default Configuration'>: 0.7407407407407407,
 <ExecutionConfiguration.T2: 'With T2'>: 0.8518518518518519,
 <ExecutionConfiguration.FLAIR: 'With FLAIR'>: 0.8518518518518519}

In [6]:
_ = joblib.dump(models, "flaml.joblib")

In [2]:
models = joblib.load("flaml.joblib")

In [32]:
default_automl.best_loss_per_estimator

{'lgbm': 0.1415242165242165,
 'rf': 0.2632834757834758,
 'xgboost': 0.13368945868945864,
 'extra_tree': 0.2409544159544159,
 'xgb_limitdepth': 0.1514245014245014,
 'lrl1': 0.11602564102564103}

In [29]:
default_model = models["DEFAULT"]
default_automl = default_model.steps[1][1]
default_automl.model.estimator

LGBMClassifier(colsample_bytree=0.8586626338818925,
               learning_rate=0.7015361546224489, max_bin=255,
               min_child_samples=41, n_estimators=45, n_jobs=16, num_leaves=6,
               reg_alpha=0.0013361660550506697, reg_lambda=0.007699408191396938,
               verbose=-1)

In [30]:
default_model = models["FLAIR"]
default_automl = default_model.steps[1][1]
default_automl.model.estimator

LogisticRegression(C=4.574851752082774, n_jobs=16, penalty='l1', solver='saga')

In [31]:
default_model = models["T2"]
default_automl = default_model.steps[1][1]
default_automl.model.estimator

LogisticRegression(C=4.803836273065365, n_jobs=16, penalty='l1', solver='saga')